# Spectral Resolution & Efficiency vs Grism Resolvance Trade
Tradeoff between spectral resolution and effiency vs grism resolvance.

In [43]:
# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from payload_designer import components, systems
from payload_designer.libs import utillib

## Setup
Run this section onwards when a parameter is updated.

### Parameters

In [44]:
# constants
target_wavelength = 1650 * unit.nm
diameter = 100 * unit.mm
upper_wavelength = 1700 * unit.nm
lower_wavelength = 900 * unit.nm
line_density = 1 * unit.mm
optical_beam_diameter = 1 * unit.mm
skew_angle = np.array([0, 0]) * unit.deg
slit_size = np.array([3, 1]) * unit.mm
fringe_frequency = 600 * (1 / unit.mm)


# variables
plot_start = 10
plot_stop = 100
plot_step = 1
focal_length = np.arange(start=plot_start, stop=plot_stop, step=plot_step) * unit.mm
beam_diameter = np.arange(start=plot_start, stop=plot_stop, step=plot_step) * unit.mm
grism_resolvance = np.arange(start=plot_start, stop=plot_stop, step=plot_step) * unit.mm
fringe_count = np.arange(start=plot_start, stop=plot_stop, step=plot_step) * unit.mm


# vectorization
parameter_space_shape = (focal_length.size, slit_size.size)

focal_length = utillib.orient_and_broadcast(
    a=focal_length, dim=0, shape=parameter_space_shape
)

### Instantiation

In [45]:
# components
sensor = components.sensors.TauSWIR()
foreoptic = components.foreoptics.Foreoptic(
    diameter=diameter, focal_length=focal_length
)
slit = components.masks.RectSlit(size=slit_size)
sr_grating = components.diffractors.SRTGrating(fringe_frequency=fringe_frequency)
# systems
payload = systems.payloads.HyperspectralImager(
    sensor=sensor, foreoptic=foreoptic, slit=slit, diffractor=sr_grating
)

fringe_count = sr_grating.get_illuminated_fringe_count(beam_diameter=beam_diameter)

## Pipeline

In [46]:
sensor_spectral_resolution = payload.get_spectral_resolution(
    upper_wavelength=upper_wavelength,
    lower_wavelength=lower_wavelength,
    target_wavelength=target_wavelength,
    beam_diameter=beam_diameter,
)

print(f"Sensor-limited spectral resolution shape: {sensor_spectral_resolution.size}")

# upper_wavelength, lower_wavelength, n_biny, n_px_y

Sensor-limited spectral resolution shape: 90


## Plots

In [47]:
data_resolvance = {
    f"Grism Resolvance": grism_resolvance[:],
    f"Spectral Resolution (along-track) [{sensor_spectral_resolution.unit}]": sensor_spectral_resolution[
        :
    ],
}


data_fringe = {
    f"Fringe Count": fringe_count[:],
    f"Spectral Resolution (along-track) [{sensor_spectral_resolution.unit}]": sensor_spectral_resolution[
        :
    ],
}

df_resolvance = pd.DataFrame.from_dict(data=data_resolvance)
df_fringe = pd.DataFrame.from_dict(data=data_fringe)


graph_resolvance = plot.line(
    df=df_resolvance,
    x=f"Grism Resolvance",
    y=[f"Spectral Resolution (along-track) [{sensor_spectral_resolution.unit}]"],
    title="FINCH Eye Spectral Resolution vs Grism Resolvance",
    dark=True,
)

graph_fringe = plot.line(
    df=df_fringe,
    x=f"Fringe Count",
    y=[f"Spectral Resolution (along-track) [{sensor_spectral_resolution.unit}]"],
    title="FINCH Eye Spectral Resolution vs Fringe Count",
    dark=True,
)

graph_resolvance.show()
graph_fringe.show()

display(df_resolvance)
display(df_fringe)

,Grism Resolvance,Spectral Resolution (along-track) [nm]
0,10.0,0.275000
1,11.0,0.250000
2,12.0,0.229167
3,13.0,0.211538
4,14.0,0.196429
...,...,...
85,95.0,0.156250
86,96.0,0.156250
87,97.0,0.156250
88,98.0,0.156250


,Fringe Count,Spectral Resolution (along-track) [nm]
0,6000.0,0.275000
1,6600.0,0.250000
2,7200.0,0.229167
3,7800.0,0.211538
4,8400.0,0.196429
...,...,...
85,57000.0,0.156250
86,57600.0,0.156250
87,58200.0,0.156250
88,58800.0,0.156250
